In [21]:
import pandas as pd
import numpy as np
import torch
import pickle
import xgboost as xgb
from importlib import reload
import synthesizer,transformer,sampler,conditional,models

In [2]:
INPUT_FILE_PATH = "/Users/sahanalva/Downloads/application_train.csv" 
SAVE_PROCESSED_ORIGNAL_DATA_PATH = "/Users/sahanalva/Counterfactual Research/home_credit_sample_processed_data.csv"
SAVE_MODEL_PATH = "/Users/sahanalva/Counterfactual Research/home_credit_sample.pt"
BLACK_BOX_PATH = "/Users/sahanalva/Counterfactual Research/tree_model_test.dat"

In [3]:
df = pd.read_csv(INPUT_FILE_PATH) 

In [4]:
# required_cols = ['loan_type_name','loan_purpose_name','agency_abbr','property_type_name','state_name','preapproval_name', 'purchaser_type_name','loan_amount_000s', 'tract_to_msamd_income', 'population','number_of_1_to_4_family_units','minority_population','applicant_income_000s','hud_median_family_income','number_of_owner_occupied_units']
# conditional_cols = None

In [5]:
def convert_to_string(df, discrete_columns):
    for col in discrete_columns:
        df[col] = df[col].astype(str)
    return df

In [6]:
discrete_columns = ['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE']
#  'NAME_EDUCATION_TYPE',
#  'NAME_FAMILY_STATUS',
#  'NAME_HOUSING_TYPE',
#  'OCCUPATION_TYPE',
#  'WEEKDAY_APPR_PROCESS_START',
#  'ORGANIZATION_TYPE', 
#  'CNT_CHILDREN',
#  'CNT_FAM_MEMBERS',
#  'REG_REGION_NOT_LIVE_REGION']

numeric_columns= ['AMT_INCOME_TOTAL',
        'AMT_CREDIT',
        'AMT_ANNUITY',
        'EXT_SOURCE_1',
        'EXT_SOURCE_2',
        'EXT_SOURCE_3']
        #'DAYS_LAST_PHONE_CHANGE',
        # 'AMT_GOODS_PRICE',
        # 'REGION_POPULATION_RELATIVE',
        # 'DAYS_BIRTH',
        # 'DAYS_EMPLOYED',
        # 'DAYS_REGISTRATION',
        # 'DAYS_ID_PUBLISH']

required_cols = discrete_columns + numeric_columns
conditional_cols = None

In [7]:
df = df[required_cols]
df[numeric_columns] = df[numeric_columns].fillna(0)
df[discrete_columns] = df[discrete_columns].fillna("NA")
df = convert_to_string(df,discrete_columns)
#df = _factorize_categoricals(df, discrete_columns)


In [8]:
# df = df[required_cols]
# nunique = df.apply(pd.Series.nunique)
# discrete_columns = nunique[nunique < 100].index
# numeric_columns = df.select_dtypes(include=['number']).columns
# object_columns = df.select_dtypes(include=['object']).columns
# df[numeric_columns] = df[numeric_columns].fillna(0)
# df[object_columns] = df[object_columns].fillna("NA")

df.isna().any()

NAME_CONTRACT_TYPE            False
CODE_GENDER                   False
FLAG_OWN_CAR                  False
FLAG_OWN_REALTY               False
NAME_TYPE_SUITE               False
NAME_INCOME_TYPE              False
NAME_EDUCATION_TYPE           False
NAME_FAMILY_STATUS            False
NAME_HOUSING_TYPE             False
OCCUPATION_TYPE               False
WEEKDAY_APPR_PROCESS_START    False
ORGANIZATION_TYPE             False
CNT_CHILDREN                  False
CNT_FAM_MEMBERS               False
REG_REGION_NOT_LIVE_REGION    False
AMT_INCOME_TOTAL              False
AMT_CREDIT                    False
AMT_ANNUITY                   False
AMT_GOODS_PRICE               False
REGION_POPULATION_RELATIVE    False
DAYS_BIRTH                    False
DAYS_EMPLOYED                 False
DAYS_REGISTRATION             False
DAYS_ID_PUBLISH               False
EXT_SOURCE_1                  False
EXT_SOURCE_2                  False
EXT_SOURCE_3                  False
DAYS_LAST_PHONE_CHANGE      

In [12]:
df = df[0:100000]
model = synthesizer.CTGANSynthesizer(embedding_dim = 128, batch_size=500)
model.fit(df, 1, BLACK_BOX_PATH, discrete_columns, conditional_cols, 200,log_frequency= False)

1.2572665214538574, Conditional Loss:0.8849444389343262, Flip Loss:1.1966543197631836
Base Loss:1.1964364051818848, Conditional Loss:4.2554612159729, Flip Loss:1.18965744972229
Base Loss:1.2571181058883667, Conditional Loss:3.814457893371582, Flip Loss:1.1864235401153564
Base Loss:1.2745620012283325, Conditional Loss:3.43792462348938, Flip Loss:1.1749849319458008
Base Loss:1.3188954591751099, Conditional Loss:2.8224329948425293, Flip Loss:1.1692992448806763
Base Loss:1.1527653932571411, Conditional Loss:2.213632345199585, Flip Loss:1.1815696954727173
Base Loss:1.2469969987869263, Conditional Loss:4.868907451629639, Flip Loss:1.1761478185653687
Condtional Cross Entropy Loss tensor(4.8689, grad_fn=<DivBackward0>)
Epoch 199, Loss G: 7.2921, Loss D: -0.1936, Loss Conditional D: -0.7309
Base Loss:1.1784964799880981, Conditional Loss:0.8764562010765076, Flip Loss:1.1900064945220947
Base Loss:1.1618523597717285, Conditional Loss:3.9168031215667725, Flip Loss:1.1881511211395264
Base Loss:1.147

In [37]:
sampled = model.sample(100000,col_index= [3])

tensor(2.3585, grad_fn=<DivBackward0>)
tensor(2.4237, grad_fn=<DivBackward0>)
tensor(2.6591, grad_fn=<DivBackward0>)
tensor(2.5600, grad_fn=<DivBackward0>)
tensor(2.5097, grad_fn=<DivBackward0>)
tensor(2.4086, grad_fn=<DivBackward0>)
tensor(2.3558, grad_fn=<DivBackward0>)
tensor(2.4477, grad_fn=<DivBackward0>)
tensor(2.4152, grad_fn=<DivBackward0>)
tensor(2.4922, grad_fn=<DivBackward0>)
tensor(2.3866, grad_fn=<DivBackward0>)
tensor(2.3268, grad_fn=<DivBackward0>)
tensor(2.3892, grad_fn=<DivBackward0>)
tensor(2.4448, grad_fn=<DivBackward0>)
tensor(2.4101, grad_fn=<DivBackward0>)
tensor(2.3630, grad_fn=<DivBackward0>)
tensor(2.5483, grad_fn=<DivBackward0>)
tensor(2.4737, grad_fn=<DivBackward0>)
tensor(2.4480, grad_fn=<DivBackward0>)
tensor(2.2438, grad_fn=<DivBackward0>)
tensor(2.3413, grad_fn=<DivBackward0>)
tensor(2.3735, grad_fn=<DivBackward0>)
tensor(2.5830, grad_fn=<DivBackward0>)
tensor(2.4175, grad_fn=<DivBackward0>)
tensor(2.4826, grad_fn=<DivBackward0>)
tensor(2.4983, grad_fn=<D

In [14]:
torch.save(model, SAVE_MODEL_PATH)
df.to_csv(SAVE_MODEL_PATH, index = False)

In [17]:
bst = pickle.load(open("/Users/sahanalva/Counterfactual Research/tree_model_test.dat", "rb"))

In [23]:
sampled_processed = _factorize_categoricals(sampled, discrete_columns)
sampled_processed = xgb.DMatrix(data = sampled_processed)
black_box_pred_prob = bst.predict(sampled_processed)

In [19]:
def _factorize_categoricals(df, discrete_columns):
    for col in discrete_columns:
        df[col], _ = pd.factorize(df[col])
    return df 

In [27]:
sum(black_box_pred_prob < 0.5)

98946

In [38]:
sampled['CODE_GENDER'].value_counts()

M      55951
F      41929
XNA     2120
Name: CODE_GENDER, dtype: int64